# GIFT Direct Eigenvalue Computation

**Méthode directe** : Discrétisation FEM du Laplacien courbé + eigsh

Pas de PINN, pas d'optimisation - calcul matriciel direct.

In [ ]:
!pip install -q numpy scipy matplotlib
import numpy as np
from scipy.sparse import lil_matrix, csr_matrix
from scipy.sparse.linalg import eigsh
import matplotlib.pyplot as plt
print('Ready!')

In [ ]:
def build_laplacian_1d(n, h, g_func, periodic=True):
    """Build 1D curved Laplacian: (1/sqrt(g)) d/dx (sqrt(g)/g * df/dx)"""
    L = lil_matrix((n, n))
    x = np.arange(n) * h
    g = g_func(x)  # g(x) for each point
    sqrt_g = np.sqrt(g)
    
    for i in range(n):
        ip = (i + 1) % n if periodic else min(i + 1, n - 1)
        im = (i - 1) % n if periodic else max(i - 1, 0)
        
        # Coefficients for conservative discretization
        # d/dx (A(x) df/dx) where A = sqrt(g)/g = 1/sqrt(g)
        A_p = 2.0 / (sqrt_g[i] + sqrt_g[ip])  # A at i+1/2
        A_m = 2.0 / (sqrt_g[i] + sqrt_g[im])  # A at i-1/2
        
        L[i, ip] = A_p / h**2
        L[i, im] = A_m / h**2
        L[i, i] = -(A_p + A_m) / h**2
    
    return csr_matrix(L)

In [ ]:
def build_laplacian_7d_separable(n, L_total, metric_1d_funcs):
    """
    Build 7D Laplacian for SEPARABLE metric g = g1(x1) * g2(x2) * ... * g7(x7)
    
    Delta_g = sum_i Delta_i where Delta_i acts on dimension i
    
    Uses Kronecker product structure for efficiency.
    """
    h = L_total / n
    dim = 7
    N_total = n**dim
    
    # Build 1D Laplacians
    L1d_list = []
    for d in range(dim):
        L1d = build_laplacian_1d(n, h, metric_1d_funcs[d])
        L1d_list.append(L1d)
    
    # Identity matrix
    I = csr_matrix(np.eye(n))
    
    # Build 7D Laplacian via Kronecker products
    # L = L1 x I x I x ... + I x L2 x I x ... + ...
    from scipy.sparse import kron
    
    L_7d = None
    for d in range(dim):
        # Build Kronecker product: I x I x ... x L_d x ... x I
        term = csr_matrix(np.array([[1.0]]))
        for j in range(dim):
            if j == d:
                term = kron(term, L1d_list[d])
            else:
                term = kron(term, I)
        
        if L_7d is None:
            L_7d = term
        else:
            L_7d = L_7d + term
    
    return L_7d

In [ ]:
# Test 1: Flat torus (calibration)
# On flat T^7 with L=2*pi, lambda_1 = 1 (from e^{ix})
print('='*60)
print('TEST 1: Flat Torus T^7 (calibration)')
print('Expected: lambda_1 = 1.0')
print('='*60)

def flat_metric(x):
    return np.ones_like(x)

for n in [4, 5, 6]:
    L_total = 2 * np.pi
    metric_funcs = [flat_metric] * 7
    L = build_laplacian_7d_separable(n, L_total, metric_funcs)
    
    # Compute eigenvalues (smallest magnitude, but Laplacian is negative)
    # Use shift-invert for small eigenvalues
    k = 5
    evals, _ = eigsh(L, k=k, sigma=0, which='LM', tol=1e-8)
    evals = -np.sort(-evals)  # Sort descending (most negative first)
    lambda_1 = -evals[1]  # First non-zero (evals[0] ~ 0)
    
    print(f'n={n}: lambda_1 = {lambda_1:.6f}, error = {abs(lambda_1-1)*100:.2f}%')

In [ ]:
# Test 2: Scaled torus (sanity check)
# If g_i = c^2, then lambda_1 = 1/c^2
# For c^2 = H*/14, lambda_1 = 14/H*
print('\n' + '='*60)
print('TEST 2: Scaled Torus (sanity check)')
print('g_i = H*/14 => lambda_1 = 14/H*')
print('='*60)

for H_star in [56, 72, 99, 104]:
    c_sq = H_star / 14.0
    
    def scaled_metric(x, c2=c_sq):
        return c2 * np.ones_like(x)
    
    n = 5
    L_total = 2 * np.pi
    metric_funcs = [lambda x, c2=c_sq: c2 * np.ones_like(x)] * 7
    L = build_laplacian_7d_separable(n, L_total, metric_funcs)
    
    evals, _ = eigsh(L, k=5, sigma=0, which='LM', tol=1e-8)
    evals = -np.sort(-evals)
    lambda_1 = -evals[1]
    
    expected = 14.0 / H_star
    print(f'H*={H_star}: lambda_1={lambda_1:.6f}, 14/H*={expected:.6f}, error={abs(lambda_1-expected)/expected*100:.2f}%')

In [ ]:
# Test 3: TCS-like metric (the REAL test)
# Metric: g_0(t) = 1, g_i(y) = h(y)^2 where h = cosh(y/T0)
# This is a VARIABLE metric - not designed to give 14/H*
print('\n' + '='*60)
print('TEST 3: TCS Metric (variable, NOT designed for 14/H*)')
print('='*60)

def tcs_metric_neck(t, T, T0):
    """Neck profile: h(t)^2 where h = cosh((t-T/2)/T0)"""
    return np.cosh((t - T/2) / T0)**2

def tcs_metric_flat(x):
    return np.ones_like(x)

results_tcs = []

for name, b2, b3 in [('K7', 21, 77), ('J1', 12, 43), ('J4', 0, 103), ('Kov', 0, 71)]:
    H_star = b2 + b3 + 1
    T = np.sqrt(H_star)
    T0 = T / 3
    
    # Neck direction has TCS profile, others are flat
    def neck_metric(x, T=T, T0=T0):
        return tcs_metric_neck(x, T, T0)
    
    # Domain: [0, T] for neck, [0, 1] for others
    # But for separable, we use [0, 2*pi] and scale metrics
    L_total = 2 * np.pi
    
    # Metric functions - first is neck, others flat
    # Scale neck metric by (T/L_total)^2 to account for domain scaling
    scale = (T / L_total)**2
    metric_funcs = [
        lambda x, s=scale, T=T, T0=T0: s * tcs_metric_neck(x * T / L_total, T, T0),
        tcs_metric_flat, tcs_metric_flat, tcs_metric_flat,
        tcs_metric_flat, tcs_metric_flat, tcs_metric_flat
    ]
    
    n = 5
    L = build_laplacian_7d_separable(n, L_total, metric_funcs)
    
    evals, _ = eigsh(L, k=5, sigma=0, which='LM', tol=1e-8)
    evals = -np.sort(-evals)
    lambda_1 = -evals[1]
    
    expected = 14.0 / H_star
    product = lambda_1 * H_star
    
    print(f'{name}: H*={H_star}, lambda_1={lambda_1:.6f}, 14/H*={expected:.6f}, lambda_1*H*={product:.2f}')
    results_tcs.append((name, H_star, lambda_1, expected, product))

In [ ]:
# Summary
print('\n' + '='*60)
print('SUMMARY: TCS Metric Results')
print('='*60)
print(f"{'Manifold':<10} {'H*':>5} {'lambda_1':>10} {'14/H*':>10} {'lambda_1*H*':>12}")
print('-'*50)
products = []
for name, H_star, lam, exp, prod in results_tcs:
    print(f'{name:<10} {H_star:>5} {lam:>10.6f} {exp:>10.6f} {prod:>12.2f}')
    products.append(prod)
print('-'*50)
print(f"{'Mean':<10} {'':<5} {'':<10} {'':<10} {np.mean(products):>12.2f}")
print(f"{'Std':<10} {'':<5} {'':<10} {'':<10} {np.std(products):>12.2f}")
print(f'\nTarget: lambda_1 * H* = 14')

In [ ]:
# Conclusion
mean_prod = np.mean(products)
print('\n' + '='*60)
print('CONCLUSION')
print('='*60)
print(f'\nMean lambda_1 * H* = {mean_prod:.2f}')
print(f'GIFT prediction = 14.00')
print(f'Deviation = {abs(mean_prod - 14)/14*100:.1f}%')
if abs(mean_prod - 14) < 2:
    print('\n=> STRONG validation of GIFT formula!')
elif abs(mean_prod - 14) < 5:
    print('\n=> Suggestive, but TCS model may not capture full G2 geometry')
else:
    print(f'\n=> TCS separable metric gives lambda_1*H* ~ {mean_prod:.1f}, not 14')